In [39]:
import io 
import os.path
from pathlib import Path

from googleapiclient.http import MediaIoBaseDownload
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Acessando uma service account

In [3]:
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]
SERVICE_ACCOUNT_KEY_FILE = f'{Path.home()}/workspace/.sacreds/guia-anuros_service_account_key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_KEY_FILE, SCOPES)

service = build('drive', 'v3', credentials=credentials)

In [ ]:
NOME_DIRETORIO_RAIZ = 'Guia Anuro Chapada dos Veadeiros'
NOME_DIRETORIO_ESPECIES='Perfil'

DATA_ROOT_DIR = 'dados_guia_digital'
SPECIES_DATA_TABLE = 'dados_especies'
AUTORES_DATA_TABLE = 'dados_autores'

# Busca arquivos e diretórios da raiz do projeto
project_root_id = service.files().list( q=f"name contains '{NOME_DIRETORIO_RAIZ}'" ).execute().get('files')[0].get('id')
project_root_items = service.files().list( q=f"'{project_root_id}' in parents").execute()

# Busca arquivos e diretórios no diretório de dados do guia digital
DATA_ROOT_DIR_ID = service.files().list( q=f"name contains '{DATA_ROOT_DIR}'" ).execute().get('files')[0].get('id')

### Tabela de dados das espécies

In [152]:
# Busca tabela de dados de espécies
id_speciesDataTable = service.files().list(q=f"name contains '{SPECIES_DATA_TABLE}'").execute().get('files')[0].get('id')
speciesDataTable = service.files().export_media( fileId=id_speciesDataTable, mimeType='text/csv').execute()

fname = 'speciesdata.csv'
with open (fname, 'wb') as f:
    f.write(speciesDataTable)

print(f"Done! Created file {fname} with species data.\n")

Done! Created file speciesdata.csv with species data.



### Busca páginas das espécies

In [108]:
# Busca páginas de famílias e espécies
id_articlesFolder = list(filter( lambda x: x.get('name')==NOME_DIRETORIO_ESPECIES, project_root_items.get('files')))[0].get('id')

familias_dirs = { i['name']:i['id'] for i in service.files().list(
    q=f"'{id_articlesFolder}' in parents and mimeType contains 'vnd.google-apps.folder'",
    fields='files(id,name)').execute().get('files') 
}

def getFamilyDocuments(dir_id):
    return service.files().list(
        q=f"'{dir_id}' in parents and mimeType contains 'vnd.google-apps.document'",
        fields='files(id,name)').execute().get('files')

data = [ {'id':fid, 'familia_name':f, 'paginas':getFamilyDocuments(fid)} for f,fid in familias_dirs.items() ]


In [ ]:
# Update species pages
basepath = '../../especies/'

for docid, species in [ (doc['id'],doc['name']) for familia in data for doc in familia.get('paginas') if doc['name'] not in ('familia')]:
    print(species)
    
    request = service.files().export_media(fileId=docid, mimeType='text/plain')
    with open(basepath+f'{species}.md', 'wb') as f:
        f.write(request.execute())

1IPhaBl8WBpLrmO795tuWURNtr_QDqhAwTHZhDUZjDMA
1LWdknpKD9fdK9T3wh68Zm-WaAcUGodcCYsyAY_GsiSs
1u3vMQX791BG9M6mZtnVtsld0MRU2St0q7_58UwpRd1Q
1928DoysFGP7G-e6QPfUBpLIG5F0R_UX3uAUZNXPgvVw
12xClS3JQA04YmJPMjbGHASxLDLM3d7v5QCi5124BeZ4
1McQafkqPmfYXVbAb33KAN7i2V7AWuVQLQwGpAcLBALk
1T48xmn-oWCA3UqhSH_nDHXDwpfO8FCfIiI4RRB1akNw
1lt0FxGhranN1cLNl4DIhRwE1SV5qxCZEbRITn9GqmBk
1CVN12G9ddJ2tuO38D8WynKwFbi9nAc-jHx4ihncxYD4
1aaCit7fY4nhXs0xTGGUne8V5Sh1xENdfoV1OnhZlP2k
19dlvLGOSDnKIn4sYt1lU0XMF4ZRFqgTOPBPZEz5LtjA
1H_guNODI2Gox_sUQUb-uIoZY6HLFeFHE7NBIvXVm_fY
1kAN4vcsAsI-hY2xRlHZxi6h6Zd5zApJ155yKDIHaAKg
1KXRP7oIDpPUo0fOslwYdUAQrLrGrSO8E2qLb4Y7OJqk
1uIuVulOnMZxGtL0aC7X5BmeXWVpmkEWhXgt3dNCT5xA
1Xw130PXOTB7sYfJkEOnhr32KqbWz-ke1_1bT-4ocnQk
1VsNgvBxiItoeJ01E343IejcdH78rXw8Wz2IGSYWzkh4
1abOSUiEC47a2ePSQSAZKUN8T0vSHpCnC5EsN_BuqWRk
14UvTMsrrOOUn0iG0bfDfewllOZbvy_yWa_TZfcRbjFM
19gATfsSJm0x32dlWUjpWisR8u0rpH7X9StIayS6JHno
1nSU9n39T009ItDK-CBRVs5eYV7g8ZVK3Ge1Sjst5aME
1zC1CbE_7mhae2xSlXjkwSkotYrXBzklGfyIX3BFDQCc
1wagxrLSfP

In [132]:
# Update families pages
basepath = '../../familias/'

for docid, family in [ (doc['id'],familia['familia_name']) for familia in data for doc in familia.get('paginas') if doc['name']=='familia' ]:
    print(family)
    request = service.files().export_media(fileId=docid, mimeType='text/plain')
    with open(basepath+f'{family}.md', 'wb') as f:
        f.write(request.execute())

siphonopidae
odontophrynidae
microhylidae
leptodactylidae
hylidae
dendrobatidae
craugastoridae
bufonidae
aromobatidae


### Tabela de autores

In [70]:
# Busca a planilha (tabela de autores) e salva como csv 

id_autoresDataTable = service.files().list( q=f"'{DATA_ROOT_DIR_ID}' in parents and name contains '{AUTORES_DATA_TABLE}' and mimeType contains 'vnd.google-apps.spreadsheet'").execute()['files'][0].get('id')
autoresDataTable = service.files().export_media( fileId=id_autoresDataTable, mimeType='text/csv').execute()

fname = 'autoresdata.csv'
with open (fname, 'wb') as f:
    f.write(autoresDataTable)

print(f"Done! Created file {fname}.\n")

Done! Created file autoresdata.csv.



### Download imagens (avatar) dos autores

In [69]:
id_autoresAvatarDir = service.files().list(
    q=f"'{DATA_ROOT_DIR_ID}' in parents and name contains 'autores_avatar' and mimeType contains 'vnd.google-apps.folder'"
).execute()['files'][0].get('id')

autoresAvatars = service.files().list(
    q=f"'{id_autoresAvatarDir}' in parents"
).execute()['files']


basepath = '../../assets/img/authors-avatar/'
for nome, fileId in [ (a.get('name'), a.get('id')) for a in autoresAvatars ]:

    print(f"Download da imagem {nome}")
    request = service.files().get_media(fileId=fileId)
    fh = io.FileIO(basepath+nome, 'wb') # this can be used to write to disk
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

Download da imagem acholler.jpg
Download 100%.
Download da imagem carlos.jpg
Download 100%.
Download da imagem fvs.jpg
Download 100%.
Download da imagem lafuc.jpg
Download 100%.
Download da imagem pholiveira.jpg
Download 100%.
Download da imagem petbio.jpg
Download 100%.
Download da imagem reuber.jpg
Download 100%.
Download da imagem jmabreu.jpg
Download 100%.
Download da imagem bvasconcelos.jpg
Download 100%.
Download da imagem ysoares.jpg
Download 100%.
Download da imagem pcsiracusa.jpg
Download 100%.


In [ ]:
basepath = '../../assets/img/authors-avatar'
basepath+nome

'../../especies/jmabreu.jpg'

In [47]:
service.files().export(fileId=autoresAvatars[1], mimeType='image/jpeg').execute()

HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/%7B%27kind%27%3A%20%27drive%23file%27%2C%20%27mimeType%27%3A%20%27image%2Fjpeg%27%2C%20%27id%27%3A%20%271yn2d83HNPWLUOAzJ5KOvRt6otGOCU595%27%2C%20%27name%27%3A%20%27fvs.jpg%27%7D/export?mimeType=image%2Fjpeg returned "File not found: {'kind': 'drive#file', 'mimeType': 'image/jpeg', 'id': '1yn2d83HNPWLUOAzJ5KOvRt6otGOCU595', 'name': 'fvs.jpg'}.". Details: "[{'message': "File not found: {'kind': 'drive#file', 'mimeType': 'image/jpeg', 'id': '1yn2d83HNPWLUOAzJ5KOvRt6otGOCU595', 'name': 'fvs.jpg'}.", 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">